In [8]:
import numpy as np

# import pyplot and set some parameters to make plots prettier
import matplotlib.pyplot as plt
from code.plot_utils import plot_pretty
plot_pretty()
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
from code.halo_shape_calc import quad_moment
from code.lightcone_query_ra_dec import query_file, read_radial_bin
from code.setup.setup import data_home_dir
import pyfits
import healpy as hp
datadir = data_home_dir()


In [32]:
"""
Follow convention of Ken Osato: Use reduced quadropole moment to find axis ratio of ellipsoidal cluster
1. Project onto principle axes spitted out by quadropole tensor
2. Do not remove particles. Particles chosen for those inside Rvir
3. Use Reduced tensor
4. q, s refer to ratio of minor to major, and intermediate to major axis

Returns:
converge -- Boolean
[a,b,c] -- normalized major, intermediate, minor axes lengths (only ratio matters in reduced tensor)
[lx, ly, lz] -- direction of minor, intermediate, major (NOT WORKING YET, they output [1,0,0], [0,1,0],[0,0,1])
"""

def quad_moment(ptcl_coord, centr, rvir):
    centr_x = centr[0]; centr_y = centr[1]; centr_z = centr[2]
    ptcl_coord_x = ptcl_coord[0]; ptcl_coord_y = ptcl_coord[1]; ptcl_coord_z = ptcl_coord[2]

    rx = ptcl_coord_x - centr_x; ry = ptcl_coord_y - centr_y; rz = ptcl_coord_z - centr_z 

    R_range = np.sqrt(rx**2. + ry**2. + rz**2.)
    #rmax = np.sqrt(np.max(r_mem_ptcl[:,3]))
    #print "Number of particles before selection is ", len(rx)
    
    #Choose particles inside Rvir
    ptcl_range = np.where(R_range < rvir)
    rx = rx[ptcl_range]; ry = ry[ptcl_range]; rz = rz[ptcl_range]
     
    num_mem_ptcl = len(rx)
    #print "Number of particles inside virial radius is ", num_mem_ptcl

    #Building quadrupole tensor. 
    Rp = np.sqrt(rx**2. + ry**2. + rz**2.)
    r = np.matrix([rx,ry,rz])
    r_rdu = r/Rp
    M_rdu = r_rdu*r_rdu.T #Initial quadrupole tensor before iteration

    #Finding eigvec, eigval
    M_eigval, M_eigvec = np.linalg.eig(M_rdu)
    sort_eigval = np.argsort(M_eigval)[::-1]
    a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl) #a, b, c major, intermediate, minor
    lx, ly, lz = M_eigvec.T[sort_eigval][::-1] #lx, ly, lz minor, intermediate, major (order reversed from a, b, c)
    lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]
    
    #Sanity check
    """
    print "r_rdu", r_rdu
    check_eig = M_rdu.dot(lx) - num_mem_ptcl*c**2.*lx
    print "M_rdu.dot(lx) ", np.dot(np.array(M_rdu), lx)
    print "check_eig ", check_eig
    print "lx is ", lx
    print "M_eigvec.T[sort_eigval], ", M_eigvec.T[sort_eigval]
    print "M_eigvec[:,0] ", M_eigvec[:,0]
    print "M_eigvec[sort_eigval] ", M_eigvec[sort_eigval]
    print "M_eigvec", M_eigvec
    print "sort_eigval ", sort_eigval
    """
    
    #Initial conditions
    q_prev = 1.; s_prev = 1.
    converge = False
    conv_iter = 0

    P_tot = np.eye(3) #the multiplicative product of all projections done over each iteration
    while (not converge) & (conv_iter < 100):
        #Change of basis
        P_axis = np.matrix([lx,ly,lz])
        P_tot = P_axis*P_tot
        r_proj = P_axis*r
        rx = np.array(r_proj[0,:])[0]; ry = np.array(r_proj[1,:])[0]; rz = np.array(r_proj[2,:])[0]

        #New iteration
        q_cur = c/a; s_cur = b/a #Osato conventaion
        Rp = np.sqrt((rx/q_cur)**2. + (ry/s_cur)**2. + rz**2.)
        r = np.matrix([rx, ry, rz])
        r_rdu = r/Rp
        M_rdu = r_rdu*r_rdu.T
        M_eigval, M_eigvec = np.linalg.eig(M_rdu)
        sort_eigval = np.argsort(M_eigval)[::-1]
        a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl)
        lx, ly, lz = M_eigvec.T[sort_eigval][::-1]
        lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]
        
        #test converge
        conv_err = 1e-6
        conv_s = np.abs(1 - s_cur/s_prev); conv_q = np.abs(1 - q_cur/q_prev)
        converge = (conv_s < conv_err) & (conv_q < conv_err)
        #print "Conv_s, conv_q ", conv_s, conv_q
        #print "Number of particles ", len(rx)
        #print "a, b, c ", a, b, c
        #print "q, s are ", q_cur, s_cur  
        #print "lx", lx
        #print 'converge is ', converge
        #print '\n'
        conv_iter += 1
        q_prev = q_cur; s_prev = s_cur
    
    #find lx, ly, lz in original basis
    P_inv = np.linalg.inv(P_tot)
    l_new_basis = np.matrix([lx,ly,lz]).T
    l_orig_basis = np.transpose(P_inv*l_new_basis)
    lx_orig = l_orig_basis[0]; ly_orig = l_orig_basis[1]; lz_orig = l_orig_basis[2]
    
    
    return converge, [a,b,c], [lx_orig, ly_orig, lz_orig]

In [33]:
from code.Heidi_read_halo_particles import read_halo_ptcl
#Test the orientation of 
#Reading a sample file. Figuring out file number
ra = 64.3599024; dec = 16.68787569;
x_cen = 273.26001; y_cen = 569.31482; z_cen = 189.31299
red_cen = 0.23191588
chi_cen = np.sqrt(x_cen**2 + y_cen**2 + z_cen**2)
rbin = int(chi_cen//25)
rvir = 2.549908

path = '{}snapshot_Lightcone_{}_0'.format(datadir, rbin)    
hdr, idx = read_radial_bin(path)
nside = hdr[2]

pix = hp.ang2pix(nside, (90-dec)*np.pi / 180., ra * np.pi / 180., nest=True)

filename = 'snapshot_Lightcone_{}_{}'.format(rbin, pix)
print 'nside, pix, comv-distance, rbin ', nside, pix, chi_cen, rbin
print 'Filename is', filename

filename = datadir + 'snapshot_Lightcone_26_1'
hdr, idx, pos, vel, ptcl_ID = read_radial_bin(filename, read_pos=True, read_vel=True, read_ids=True)
npart = len(pos)//3
pos=pos.reshape(npart, 3); vel = vel.reshape(npart,3)
#print 'Position \n', pos
#print 'Velocity \n', vel
#print 'Particle ID \n', ptcl_ID

#To do: Find orientation of this halo, and scale this up to all redMapper selected halos 
#Need shape algorithm, redMapper selected Halo data, 

x, y, z = read_halo_ptcl(ra, dec, red_cen, x_cen, y_cen, z_cen, Rmax=rvir)
ptcl_coord = [x,y,z]; centr = [x_cen, y_cen, z_cen]
#print ptcl_coord[0], centr[0], rvir
#print ptcl_coord, centr
converge, axes_len, axes_dir = quad_moment(ptcl_coord, centr, rvir)

nside, pix, comv-distance, rbin  2 0 659.2645944683915 26
Filename is snapshot_Lightcone_26_0
axes length are [0.5773504279935158, 0.31355738012861206, 0.23943732103822274]
axes dir are [matrix([[ 0.73342967,  0.67975759, -0.00324522]]), matrix([[ 0.00301255, -0.00802432, -0.99996327]]), matrix([[ 0.67975866, -0.73339295,  0.00793308]])]


In [29]:
#Checking that your code makes sense

#Arbitrary rotation. Check that inverted produced same vectors
rot = np.matrix([[0.7,-1,3.2],[1.5,6.6,3],[2.1,5,1]])

#directions of vectors
va = np.array([1,-0.5,2])
vb = np.array([1,0,0])
vc = np.array([0,0,1])
vec_arr = np.matrix([va,vb,vc]).T

#Rotated vectors
P_proj = rot
P_inv = np.linalg.inv(P_proj)
vec_arr_rot = P_proj*vec_arr
vec_arr = P_inv*vec_arr_rot
va_rot = vec_arr_rot.T[0]; vb_rot = vec_arr_rot.T[1]; vc_rot = vec_arr_rot.T[2]
va = vec_arr.T[0]; vb = vec_arr.T[1]; vc = vec_arr.T[2]

After projecting and inverting vectors are  [[ 1.  -0.5  2. ]] [[1.00000000e+00 2.97422566e-17 1.85659686e-17]] [[-2.22044605e-16 -1.38777878e-17  1.00000000e+00]]
After projecting the vectors are  [[7.6 4.2 1.6]] [[0.7 1.5 2.1]] [[3.2 3.  1. ]]
